In [1]:
#Purpose of this notebook is to load results from experiment 1 for : 
# 1. - BPW + Max Freq features , TSF Minimal
# 2. - Connectivity metrics, Ada Boost for all 
# 3. - PLI + (BPW + Max Freq features, TSF Minimal)

# After loading and viewing the results I can calculate appropriate mean and s.d. values
# I can also do statistical significance tests

In [2]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


#### Loading data from Min Freq + BPW experiments

In [47]:
save_folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Combine_Max_Freq_Power_Band/'
data_types = ['REM', 'N1', 'N2', 'N3', 'Wake']

file_name = data_type + 'combined_max_freq_plus_BPW_TS_Fresh_Minimal_feats_results.pkl'
results_df = joblib.load(save_folder + file_name)[1][data_type]['regions_combined']
results_df = results_df[ ['random_state', 'y_true'] + [col for col in results_df.columns if 'Ada' in col] ]

display(results_df)

res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

res_df

,random_state,y_true,Prefrontal_Frontal_Ada_B_y_preds,Prefrontal_Left Frontal_Ada_B_y_preds,Prefrontal_Right Frontal_Ada_B_y_preds,Prefrontal_Central_Ada_B_y_preds,Prefrontal_Left Central_Ada_B_y_preds,Prefrontal_Right Central_Ada_B_y_preds,Prefrontal_Left Temporal_Ada_B_y_preds,Prefrontal_Right Temporal_Ada_B_y_preds,Prefrontal_Parietal_Ada_B_y_preds,Prefrontal_Left Parietal_Ada_B_y_preds,Prefrontal_Right Parietal_Ada_B_y_preds,Prefrontal_Occipital_Ada_B_y_preds,Frontal_Left Frontal_Ada_B_y_preds,Frontal_Right Frontal_Ada_B_y_preds,Frontal_Central_Ada_B_y_preds,Frontal_Left Central_Ada_B_y_preds,Frontal_Right Central_Ada_B_y_preds,Frontal_Left Temporal_Ada_B_y_preds,Frontal_Right Temporal_Ada_B_y_preds,Frontal_Parietal_Ada_B_y_preds,Frontal_Left Parietal_Ada_B_y_preds,Frontal_Right Parietal_Ada_B_y_preds,Frontal_Occipital_Ada_B_y_preds,Left Frontal_Right Frontal_Ada_B_y_preds,Left Frontal_Central_Ada_B_y_preds,Left Frontal_Left Central_Ada_B_y_preds,Left Frontal_Right Central_Ada_B_y_preds,Left Frontal_Left Temporal_Ada_B_y_preds,Left Frontal_Right Temporal_Ada_B_y_preds,Left Frontal_Parietal_Ada_B_y_preds,Left Frontal_Left Parietal_Ada_B_y_preds,Left Frontal_Right Parietal_Ada_B_y_preds,Left Frontal_Occipital_Ada_B_y_preds,Right Frontal_Central_Ada_B_y_preds,Right Frontal_Left Central_Ada_B_y_preds,Right Frontal_Right Central_Ada_B_y_preds,Right Frontal_Left Temporal_Ada_B_y_preds,Right Frontal_Right Temporal_Ada_B_y_preds,Right Frontal_Parietal_Ada_B_y_preds,Right Frontal_Left Parietal_Ada_B_y_preds,Right Frontal_Right Parietal_Ada_B_y_preds,Right Frontal_Occipital_Ada_B_y_preds,Central_Left Central_Ada_B_y_preds,Central_Right Central_Ada_B_y_preds,Central_Left Temporal_Ada_B_y_preds,Central_Right Temporal_Ada_B_y_preds,Central_Parietal_Ada_B_y_preds,Central_Left Parietal_Ada_B_y_preds,Central_Right Parietal_Ada_B_y_preds,Central_Occipital_Ada_B_y_preds,Left Central_Right Central_Ada_B_y_preds,Left Central_Left Temporal_Ada_B_y_preds,Left Central_Right Temporal_Ada_B_y_preds,Left Central_Parietal_Ada_B_y_preds,Left Central_Left Parietal_Ada_B_y_preds,Left Central_Right Parietal_Ada_B_y_preds,Left Central_Occipital_Ada_B_y_preds,Right Central_Left Temporal_Ada_B_y_preds,Right Central_Right Temporal_Ada_B_y_preds,Right Central_Parietal_Ada_B_y_preds,Right Central_Left Parietal_Ada_B_y_preds,Right Central_Right Parietal_Ada_B_y_preds,Right Central_Occipital_Ada_B_y_preds,Left Temporal_Right Temporal_Ada_B_y_preds,Left Temporal_Parietal_Ada_B_y_preds,Left Temporal_Left Parietal_Ada_B_y_preds,Left Temporal_Right Parietal_Ada_B_y_preds,Left Temporal_Occipital_Ada_B_y_preds,Right Temporal_Parietal_Ada_B_y_preds,Right Temporal_Left Parietal_Ada_B_y_preds,Right Temporal_Right Parietal_Ada_B_y_preds,Right Temporal_Occipital_Ada_B_y_preds,Parietal_Left Parietal_Ada_B_y_preds,Parietal_Right Parietal_Ada_B_y_preds,Parietal_Occipital_Ada_B_y_preds,Left Parietal_Right Parietal_Ada_B_y_preds,Left Parietal_Occipital_Ada_B_y_preds,Right Parietal_Occipital_Ada_B_y_preds
0,1,"[[0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, ...","[[1, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, ...","[[1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, ...","[[1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, ...","[[0, 1, 1, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 0, ...","[[0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1, ...","[[1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, ...","[[1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[1, 1, 0, 0, 1, 0, 1, 0], [1, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 1, 1], [0, 1, 0, 1, 1, 0, ...","[[1, 1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, ..."

,random_state,Prefrontal_Frontal_Ada_B_acc,Prefrontal_Frontal_Ada_B_std,Prefrontal_Left Frontal_Ada_B_acc,Prefrontal_Left Frontal_Ada_B_std,Prefrontal_Right Frontal_Ada_B_acc,Prefrontal_Right Frontal_Ada_B_std,Prefrontal_Central_Ada_B_acc,Prefrontal_Central_Ada_B_std,Prefrontal_Left Central_Ada_B_acc,Prefrontal_Left Central_Ada_B_std,Prefrontal_Right Central_Ada_B_acc,Prefrontal_Right Central_Ada_B_std,Prefrontal_Left Temporal_Ada_B_acc,Prefrontal_Left Temporal_Ada_B_std,Prefrontal_Right Temporal_Ada_B_acc,Prefrontal_Right Temporal_Ada_B_std,Prefrontal_Parietal_Ada_B_acc,Prefrontal_Parietal_Ada_B_std,Prefrontal_Left Parietal_Ada_B_acc,Prefrontal_Left Parietal_Ada_B_std,Prefrontal_Right Parietal_Ada_B_acc,Prefrontal_Right Parietal_Ada_B_std,Prefrontal_Occipital_Ada_B_acc,Prefrontal_Occipital_Ada_B_std,Frontal_Left Frontal_Ada_B_acc,Frontal_Left Frontal_Ada_B_std,Frontal_Right Frontal_Ada_B_acc,Frontal_Right Frontal_Ada_B_std,Frontal_Central_Ada_B_acc,Frontal_Central_Ada_B_std,Frontal_Left Central_Ada_B_acc,Frontal_Left Central_Ada_B_std,Frontal_Right Central_Ada_B_acc,Frontal_Right Central_Ada_B_std,Frontal_Left Temporal_Ada_B_acc,Frontal_Left Temporal_Ada_B_std,Frontal_Right Temporal_Ada_B_acc,Frontal_Right Temporal_Ada_B_std,Frontal_Parietal_Ada_B_acc,Frontal_Parietal_Ada_B_std,Frontal_Left Parietal_Ada_B_acc,Frontal_Left Parietal_Ada_B_std,Frontal_Right Parietal_Ada_B_acc,Frontal_Right Parietal_Ada_B_std,Frontal_Occipital_Ada_B_acc,Frontal_Occipital_Ada_B_std,Left Frontal_Right Frontal_Ada_B_acc,Left Frontal_Right Frontal_Ada_B_std,Left Frontal_Central_Ada_B_acc,Left Frontal_Central_Ada_B_std,Left Frontal_Left Central_Ada_B_acc,Left Frontal_Left Central_Ada_B_std,Left Frontal_Right Central_Ada_B_acc,Left Frontal_Right Central_Ada_B_std,Left Frontal_Left Temporal_Ada_B_acc,Left Frontal_Left Temporal_Ada_B_std,Left Frontal_Right Temporal_Ada_B_acc,Left Frontal_Right Temporal_Ada_B_std,Left Frontal_Parietal_Ada_B_acc,Left Frontal_Parietal_Ada_B_std,Left Frontal_Left Parietal_Ada_B_acc,Left Frontal_Left Parietal_Ada_B_std,Left Frontal_Right Parietal_Ada_B_acc,Left Frontal_Right Parietal_Ada_B_std,Left Frontal_Occipital_Ada_B_acc,Left Frontal_Occipital_Ada_B_std,Right Frontal_Central_Ada_B_acc,Right Frontal_Central_Ada_B_std,Right Frontal_Left Central_Ada_B_acc,Right Frontal_Left Central_Ada_B_std,Right Frontal_Right Central_Ada_B_acc,Right Frontal_Right Central_Ada_B_std,Right Frontal_Left Temporal_Ada_B_acc,Right Frontal_Left Temporal_Ada_B_std,Right Frontal_Right Temporal_Ada_B_acc,Right Frontal_Right Temporal_Ada_B_std,Right Frontal_Parietal_Ada_B_acc,Right Frontal_Parietal_Ada_B_std,Right Frontal_Left Parietal_Ada_B_acc,Right Frontal_Left Parietal_Ada_B_std,Right Frontal_Right Parietal_Ada_B_acc,Right Frontal_Right Parietal_Ada_B_std,Right Frontal_Occipital_Ada_B_acc,Right Frontal_Occipital_Ada_B_std,Central_Left Central_Ada_B_acc,Central_Left Central_Ada_B_std,Central_Right Central_Ada_B_acc,Central_Right Central_Ada_B_std,Central_Left Temporal_Ada_B_acc,Central_Left Temporal_Ada_B_std,Central_Right Temporal_Ada_B_acc,Central_Right Temporal_Ada_B_std,Central_Parietal_Ada_B_acc,Central_Parietal_Ada_B_std,Central_Left Parietal_Ada_B_acc,Central_Left Parietal_Ada_B_std,Central_Right Parietal_Ada_B_acc,Central_Right Parietal_Ada_B_std,Central_Occipital_Ada_B_acc,Central_Occipital_Ada_B_std,Left Central_Right Central_Ada_B_acc,Left Central_Right Central_Ada_B_std,Left Central_Left Temporal_Ada_B_acc,Left Central_Left Temporal_Ada_B_std,Left Central_Right Temporal_Ada_B_acc,Left Central_Right Temporal_Ada_B_std,Left Central_Parietal_Ada_B_acc,Left Central_Parietal_Ada_B_std,Left Central_Left Parietal_Ada_B_acc,Left Central_Left Parietal_Ada_B_std,Left Central_Right Parietal_Ada_B_acc,Left Central_Right Parietal_Ada_B_std,Left Central_Occipital_Ada_B_acc,Left Central_Occipital_Ada_B_std,Right Central_Left Temporal_Ada_B_acc,Right Central_Left Temporal_Ada_B_std,Right Central_Right Temporal_Ada_B_acc,Right Central_Right Temporal_Ad

In [36]:
data_type

'N1'

#### Load connectivity data (mainly interested in PLI)

In [25]:
metrics = ['coh', 'cohy', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'dpli', 'wpli', 'wpli2_debiased']
len(metrics)

10

In [24]:
folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Connectivity/Connectivity_Results/Experiment_set_2/'

In [29]:
methods = ['coh', 'cohy', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'dpli', 'wpli', 'wpli2_debiased']
data_types = ['REM', 'N1', 'N2', 'N3', 'Wake']
expt_num = 1
save_folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Connectivity/Connectivity_Results/Experiment_set_2/'

#Try loading a results file
method = methods[0]
data_type = data_types[1]

load_path = save_folder + data_type + method + '_expt_' + str(expt_num) + '_' + 'raw_clfs_results.h5'

results_df = pd.read_hdf(load_path)

,random_state,y_true,delta_RF_y_preds,delta_DT_y_preds,delta_Ada_B_y_preds,delta_SVM_y_preds,theta_RF_y_preds,theta_DT_y_preds,theta_Ada_B_y_preds,theta_SVM_y_preds,alpha_RF_y_preds,alpha_DT_y_preds,alpha_Ada_B_y_preds,alpha_SVM_y_preds,sigma_RF_y_preds,sigma_DT_y_preds,sigma_Ada_B_y_preds,sigma_SVM_y_preds,beta_RF_y_preds,beta_DT_y_preds,beta_Ada_B_y_preds,beta_SVM_y_preds,gamma_RF_y_preds,gamma_DT_y_preds,gamma_Ada_B_y_preds,gamma_SVM_y_preds,all_RF_y_preds,all_DT_y_preds,all_Ada_B_y_preds,all_SVM_y_preds,all_features_RF_y_preds,all_features_DT_y_preds,all_features_Ada_B_y_preds,all_features_SVM_y_preds
0,1,"[[0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, ...","[[0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 1, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 1, 0, 0, 0, 1, 1], [1, 1, 1, 1, 1, 0, ...","[[0, 1, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, ...","[[0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, ...","[[1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 1, 0, 0, ...","[[1, 1, 0, 1, 0, 1, 0, 1], [0, 0, 1, 1, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 0, 1, 0, 0, 1, 0, 1], [1, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, ...","[[1, 1, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 1, 0, 1, 0, 1], [0, 0, 1, 0, 1, 1, ...","[[1, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ..."
1,2,"[[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, ...","[[0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 1, 1, 1], [1, 0, 0, 1, 0, 0, ...","[[0, 1, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, ...","[[0, 1, 0, 1, 1, 1, 1, 0], [0, 0, 1, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 0, 0, ...","[[1, 1, 1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 0, 1, 0, 0, 1, 1, 1], [0, 0, 1, 0, 1, 1, ...","[[1, 1, 0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 1, 0, 1, 1, 1], [1, 1, 0, 1, 1, 0, ...","[[1, 0, 0, 1, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 1, 0, 1, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 0, ..."

#### Loading Data From PLI + BPW + Max_Freq

In [42]:
# For 3. , we have the following: 

folder = 'Execute New Experiments/Connectivity/view_combined_results/bpw_plus_max_freqEfficient_connectivity_coh_alpha.pkl'

folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Connectivity/view_combined_results/'

f_names = ['bpw_plus_max_freqMinimal_connectivity_pli_gamma.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_delta.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_sigma.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_alpha.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_beta.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_theta.pkl']

In [45]:
results_dict = joblib.load(folder + f_names[0])[1]
print(results_dict.keys())
results_df = results_dict['N1']['regions_combined']
results_df = results_df[ ['random_state', 'y_true'] + [col for col in results_df.columns if 'Ada' in col] ]
results_df

dict_keys(['Wake', 'N1', 'N2', 'N3', 'REM'])


,random_state,y_true,Prefrontal_Frontal_Ada_B_y_preds,Prefrontal_Left Frontal_Ada_B_y_preds,Prefrontal_Right Frontal_Ada_B_y_preds,Prefrontal_Central_Ada_B_y_preds,Prefrontal_Left Central_Ada_B_y_preds,Prefrontal_Right Central_Ada_B_y_preds,Prefrontal_Left Temporal_Ada_B_y_preds,Prefrontal_Right Temporal_Ada_B_y_preds,Prefrontal_Parietal_Ada_B_y_preds,Prefrontal_Left Parietal_Ada_B_y_preds,Prefrontal_Right Parietal_Ada_B_y_preds,Prefrontal_Occipital_Ada_B_y_preds,Frontal_Left Frontal_Ada_B_y_preds,Frontal_Right Frontal_Ada_B_y_preds,Frontal_Central_Ada_B_y_preds,Frontal_Left Central_Ada_B_y_preds,Frontal_Right Central_Ada_B_y_preds,Frontal_Left Temporal_Ada_B_y_preds,Frontal_Right Temporal_Ada_B_y_preds,Frontal_Parietal_Ada_B_y_preds,Frontal_Left Parietal_Ada_B_y_preds,Frontal_Right Parietal_Ada_B_y_preds,Frontal_Occipital_Ada_B_y_preds,Left Frontal_Right Frontal_Ada_B_y_preds,Left Frontal_Central_Ada_B_y_preds,Left Frontal_Left Central_Ada_B_y_preds,Left Frontal_Right Central_Ada_B_y_preds,Left Frontal_Left Temporal_Ada_B_y_preds,Left Frontal_Right Temporal_Ada_B_y_preds,Left Frontal_Parietal_Ada_B_y_preds,Left Frontal_Left Parietal_Ada_B_y_preds,Left Frontal_Right Parietal_Ada_B_y_preds,Left Frontal_Occipital_Ada_B_y_preds,Right Frontal_Central_Ada_B_y_preds,Right Frontal_Left Central_Ada_B_y_preds,Right Frontal_Right Central_Ada_B_y_preds,Right Frontal_Left Temporal_Ada_B_y_preds,Right Frontal_Right Temporal_Ada_B_y_preds,Right Frontal_Parietal_Ada_B_y_preds,Right Frontal_Left Parietal_Ada_B_y_preds,Right Frontal_Right Parietal_Ada_B_y_preds,Right Frontal_Occipital_Ada_B_y_preds,Central_Left Central_Ada_B_y_preds,Central_Right Central_Ada_B_y_preds,Central_Left Temporal_Ada_B_y_preds,Central_Right Temporal_Ada_B_y_preds,Central_Parietal_Ada_B_y_preds,Central_Left Parietal_Ada_B_y_preds,Central_Right Parietal_Ada_B_y_preds,Central_Occipital_Ada_B_y_preds,Left Central_Right Central_Ada_B_y_preds,Left Central_Left Temporal_Ada_B_y_preds,Left Central_Right Temporal_Ada_B_y_preds,Left Central_Parietal_Ada_B_y_preds,Left Central_Left Parietal_Ada_B_y_preds,Left Central_Right Parietal_Ada_B_y_preds,Left Central_Occipital_Ada_B_y_preds,Right Central_Left Temporal_Ada_B_y_preds,Right Central_Right Temporal_Ada_B_y_preds,Right Central_Parietal_Ada_B_y_preds,Right Central_Left Parietal_Ada_B_y_preds,Right Central_Right Parietal_Ada_B_y_preds,Right Central_Occipital_Ada_B_y_preds,Left Temporal_Right Temporal_Ada_B_y_preds,Left Temporal_Parietal_Ada_B_y_preds,Left Temporal_Left Parietal_Ada_B_y_preds,Left Temporal_Right Parietal_Ada_B_y_preds,Left Temporal_Occipital_Ada_B_y_preds,Right Temporal_Parietal_Ada_B_y_preds,Right Temporal_Left Parietal_Ada_B_y_preds,Right Temporal_Right Parietal_Ada_B_y_preds,Right Temporal_Occipital_Ada_B_y_preds,Parietal_Left Parietal_Ada_B_y_preds,Parietal_Right Parietal_Ada_B_y_preds,Parietal_Occipital_Ada_B_y_preds,Left Parietal_Right Parietal_Ada_B_y_preds,Left Parietal_Occipital_Ada_B_y_preds,Right Parietal_Occipital_Ada_B_y_preds
0,1,"[[0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, ..."

In [46]:
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
res_df

,random_state,Prefrontal_Frontal_Ada_B_acc,Prefrontal_Frontal_Ada_B_std,Prefrontal_Left Frontal_Ada_B_acc,Prefrontal_Left Frontal_Ada_B_std,Prefrontal_Right Frontal_Ada_B_acc,Prefrontal_Right Frontal_Ada_B_std,Prefrontal_Central_Ada_B_acc,Prefrontal_Central_Ada_B_std,Prefrontal_Left Central_Ada_B_acc,Prefrontal_Left Central_Ada_B_std,Prefrontal_Right Central_Ada_B_acc,Prefrontal_Right Central_Ada_B_std,Prefrontal_Left Temporal_Ada_B_acc,Prefrontal_Left Temporal_Ada_B_std,Prefrontal_Right Temporal_Ada_B_acc,Prefrontal_Right Temporal_Ada_B_std,Prefrontal_Parietal_Ada_B_acc,Prefrontal_Parietal_Ada_B_std,Prefrontal_Left Parietal_Ada_B_acc,Prefrontal_Left Parietal_Ada_B_std,Prefrontal_Right Parietal_Ada_B_acc,Prefrontal_Right Parietal_Ada_B_std,Prefrontal_Occipital_Ada_B_acc,Prefrontal_Occipital_Ada_B_std,Frontal_Left Frontal_Ada_B_acc,Frontal_Left Frontal_Ada_B_std,Frontal_Right Frontal_Ada_B_acc,Frontal_Right Frontal_Ada_B_std,Frontal_Central_Ada_B_acc,Frontal_Central_Ada_B_std,Frontal_Left Central_Ada_B_acc,Frontal_Left Central_Ada_B_std,Frontal_Right Central_Ada_B_acc,Frontal_Right Central_Ada_B_std,Frontal_Left Temporal_Ada_B_acc,Frontal_Left Temporal_Ada_B_std,Frontal_Right Temporal_Ada_B_acc,Frontal_Right Temporal_Ada_B_std,Frontal_Parietal_Ada_B_acc,Frontal_Parietal_Ada_B_std,Frontal_Left Parietal_Ada_B_acc,Frontal_Left Parietal_Ada_B_std,Frontal_Right Parietal_Ada_B_acc,Frontal_Right Parietal_Ada_B_std,Frontal_Occipital_Ada_B_acc,Frontal_Occipital_Ada_B_std,Left Frontal_Right Frontal_Ada_B_acc,Left Frontal_Right Frontal_Ada_B_std,Left Frontal_Central_Ada_B_acc,Left Frontal_Central_Ada_B_std,Left Frontal_Left Central_Ada_B_acc,Left Frontal_Left Central_Ada_B_std,Left Frontal_Right Central_Ada_B_acc,Left Frontal_Right Central_Ada_B_std,Left Frontal_Left Temporal_Ada_B_acc,Left Frontal_Left Temporal_Ada_B_std,Left Frontal_Right Temporal_Ada_B_acc,Left Frontal_Right Temporal_Ada_B_std,Left Frontal_Parietal_Ada_B_acc,Left Frontal_Parietal_Ada_B_std,Left Frontal_Left Parietal_Ada_B_acc,Left Frontal_Left Parietal_Ada_B_std,Left Frontal_Right Parietal_Ada_B_acc,Left Frontal_Right Parietal_Ada_B_std,Left Frontal_Occipital_Ada_B_acc,Left Frontal_Occipital_Ada_B_std,Right Frontal_Central_Ada_B_acc,Right Frontal_Central_Ada_B_std,Right Frontal_Left Central_Ada_B_acc,Right Frontal_Left Central_Ada_B_std,Right Frontal_Right Central_Ada_B_acc,Right Frontal_Right Central_Ada_B_std,Right Frontal_Left Temporal_Ada_B_acc,Right Frontal_Left Temporal_Ada_B_std,Right Frontal_Right Temporal_Ada_B_acc,Right Frontal_Right Temporal_Ada_B_std,Right Frontal_Parietal_Ada_B_acc,Right Frontal_Parietal_Ada_B_std,Right Frontal_Left Parietal_Ada_B_acc,Right Frontal_Left Parietal_Ada_B_std,Right Frontal_Right Parietal_Ada_B_acc,Right Frontal_Right Parietal_Ada_B_std,Right Frontal_Occipital_Ada_B_acc,Right Frontal_Occipital_Ada_B_std,Central_Left Central_Ada_B_acc,Central_Left Central_Ada_B_std,Central_Right Central_Ada_B_acc,Central_Right Central_Ada_B_std,Central_Left Temporal_Ada_B_acc,Central_Left Temporal_Ada_B_std,Central_Right Temporal_Ada_B_acc,Central_Right Temporal_Ada_B_std,Central_Parietal_Ada_B_acc,Central_Parietal_Ada_B_std,Central_Left Parietal_Ada_B_acc,Central_Left Parietal_Ada_B_std,Central_Right Parietal_Ada_B_acc,Central_Right Parietal_Ada_B_std,Central_Occipital_Ada_B_acc,Central_Occipital_Ada_B_std,Left Central_Right Central_Ada_B_acc,Left Central_Right Central_Ada_B_std,Left Central_Left Temporal_Ada_B_acc,Left Central_Left Temporal_Ada_B_std,Left Central_Right Temporal_Ada_B_acc,Left Central_Right Temporal_Ada_B_std,Left Central_Parietal_Ada_B_acc,Left Central_Parietal_Ada_B_std,Left Central_Left Parietal_Ada_B_acc,Left Central_Left Parietal_Ada_B_std,Left Central_Right Parietal_Ada_B_acc,Left Central_Right Parietal_Ada_B_std,Left Central_Occipital_Ada_B_acc,Left Central_Occipital_Ada_B_std,Right Central_Left Temporal_Ada_B_acc,Right Central_Left Temporal_Ada_B_std,Right Central_Right Temporal_Ada_B_acc,Right Central_Right Temporal_Ad